These functions will help you extract the entire report and place them into a pandas dataframe - please remember to download the conversion table (dow 30 to cik #):
1) getting_crawler(year_number, qtr): this function will download the index file as provdied by sec.gov. Please indicate the year as well the quarter you would like to extract the data from. For more information, visit https://www.sec.gov/edgar/searchedgar/accessing-edgar-data.htm

2) milestone_search(file_name): milestone_search looks for where the listing begins in the index file.

3) convert_to_pandas(file_name, data_start): convert_to_pandas function converts index file to a pandas data frame

4) search_company_name(data, company_name, report_type): allows you to find the url for the report as well as the the specified companies. Please note that the function currently only take CIK# for accuracy. Many larger companies have multiple reporting sub-companies, i.e., J P Morgan.

5) getting_data(url_list, year_number, qtr, report_type): Will download the entire webpage of the report as a file and put it into a pandas dataframe. For missing values, it will automatically assigned np.NaN

6) adding_company_name(data): will add company name and other inforamtion back to the current dataframe

7) putting_together(year_number,qtr, company_name, report_type): goes through the entire process and only return the data_frame with the full data.

8) the functions now support saving and reading in pickle files!

See below listing for the options for company_name

In [130]:
import urllib
import pandas as pd
import struct
import numpy as np
from bs4 import BeautifulSoup
import requests
import regex as re
import pickle

In [89]:
company_table = pd.read_csv("cik_list.csv",sep="\t")

In [90]:
company_table

,Company,Name,CIK #
0,AAPL,Apple Inc.,320193
1,AXP,American Expression Co.,4962
2,BA,Boeing Co.,12927
3,CAT,Caterpillar Inc.,18230
4,CSCO,Cisco Systems Inc.,858877
5,CVX,Chevron Corp,93410
6,DD,Dupont E I De Numours and Co.,30554
7,DIS,Walt Disney Co.,1001039
8,GE,General Electric Co.,40545
9,GS,Goldman Sachs Group Inc.,886982


In [120]:
def getting_crawler(year_number,qtr):
    base_url = "https://www.sec.gov/Archives/edgar/full-index/"
    year = str(year_number) + "/"
    if qtr == 1:
        QTR = "QTR1/"
    if qtr == 2:
        QTR = "QTR2/"
    if qtr == 3:
        QTR = "QTR3/"
    if qtr == 4:
        QTR = "QTR4/"
    download_file = "crawler.idx"
    full = base_url + year + QTR + download_file
    file_name = str(year_number) + "_QTR" + str(qtr) + "_crawler.idx"
    testfile = urllib.URLopener()
    testfile.retrieve(full, file_name)
    return file_name

In [121]:
def milestone_search(file_name):
    with open(file_name) as myFile:
        for num, line in enumerate(myFile, 0):
            if ("CIK" in line):
                if ("Form Type" in line):
                    #print 'found at line:', num
                    line_start = num
            if "------------------" in line:
                #print "found ----- at line:", num
                data_start = num+1
                #print "data starts at:", data_start
    return data_start

In [122]:
def convert_to_pandas(file_name, data_start):
    company_name = list()
    report_type = list()
    CIK = list()
    date = list()
    url = list()
    with open(file_name) as myfile:
        testing = myfile.readlines()[data_start:]
        for record in testing:
            record_split = map(lambda x: x.strip(), filter(lambda x: x!="" and x !="\n", record.split("  ")))
            company_name.append(record_split[0])
            report_type.append(record_split[1])
            CIK.append(record_split[2])
            date.append(record_split[3])
            url.append(record_split[4])
    data_dict = {"company":company_name,
                 "report":report_type,
                 "CIK":CIK,
                 "date":date,
                 "url":url}
    df = pd.DataFrame.from_dict(data_dict)
    df["company_l"] = df["company"].str.lower()
    return df

In [123]:
def search_company_name(data, company_name, report_type):
    company_list = dict()
    url_list = list()
    date_list = list()
    data_report = data[data["report"] == report_type]
    for company in map(lambda x: str(x), company_name):
        data_company = data_report[data_report["CIK"]==(company)]
        url_company = list(data_company["url"])
        file_date = list(data_company["date"])
        if not url_company:
            url_list.append(np.NaN)
        else:
            url_list.append(url_company[0])
        if not file_date:
            date_list.append(np.NaN)
        else:
            date_list.append(file_date[0])
    company_list["url"] = url_list
    company_list["date"] = date_list
    return company_list

In [124]:
def getting_data(company_list, year_number, qtr, report_type, company_name):
    full_text_list = list()
    url_list = company_list["url"]
    for report in url_list:
        if pd.isnull(report):
            full_text = np.NaN
            full_text_list.append(full_text)
            #print "not found"
        else:
            text = requests.get(report).text
            text = BeautifulSoup(text, "html.parser")
            target = text.find("table",{"class":"tableFile"})
            link = target.find_all("tr")[1].find_all("a")[0]["href"]
            final_link = "https://www.sec.gov"+link
            text_report = requests.get(final_link).text
            text_report = BeautifulSoup(text_report, "html.parser")
            content = text_report.text
            content = content.encode(encoding='UTF-8')
            full_text_list.append(content)
            #print"found it"
    #print len(names)
    #print len(full_text_list)
    data_text = {"CIK":company_name,"full_text":full_text_list,"url":company_list["url"], "file_date":company_list["date"]}
    df_text = pd.DataFrame.from_dict(data_text)
    df_text["year"] = year_number
    df_text["quarter"] = qtr
    df_text["report_type"] = report_type
    return df_text

In [125]:
def adding_company_name(data):
    company_table = pd.read_csv("cik_list.csv",sep="\t")
    df_return = pd.merge(data, company_table, how = "left", left_on='CIK', right_on="CIK #")
    return df_return

In [144]:
def putting_together(year_number,qtr, company_name, report_type):
    file_path = getting_crawler(year_number,qtr)
    data_start_num = milestone_search(file_path)
    data_frame = convert_to_pandas(file_path, data_start_num)
    results = search_company_name(data_frame, company_name, report_type)
    final_data = getting_data(results, year_number, qtr, report_type, company_name)
    return_file = adding_company_name(final_data)
    return return_file.dropna().reset_index(drop=True)

Saving the final_dataset to a pickle file, and reading in a pickle file

In [156]:
def save_as_pickle(path_to_save, final_data):
    final_data.to_pickle(path_to_save)

In [157]:
def read_data(path_to_open):
    infile = open(path_to_open,"rb")
    newdata = pickle.load(infile)
    infile.close()
    return newdata

Demonstration - extracting the latest document for all 30 companies

In [108]:
names = list(company_table[["CIK #"]].iloc[0:30,:]["CIK #"])

In [109]:
data_2017_qtr1 = putting_together(2017, 1, names, "10-K")

In [127]:
data_2016_qtr4 = putting_together(2016, 4, names, "10-K")

In [134]:
data_2016_qtr3 = putting_together(2016, 3, names, "10-K")

In [151]:
final = [data_2017_qtr1,data_2016_qtr4,data_2016_qtr3]
latest = pd.concat(final).reset_index(drop=True)

In [152]:
latest

,CIK,file_date,full_text,url,year,quarter,report_type,Company,Name,CIK #
0,4962,2017-02-17,\n10-K\n1\nd321397d10k.htm\nFORM 10-K\n\n\nFor...,https://www.sec.gov/Archives/edgar/data/4962/0...,2017,1,10-K,AXP,American Expression Co.,4962
1,12927,2017-02-08,\n10-K\n1\na201612dec3110k.htm\n10-K\n\n\n\n\n...,https://www.sec.gov/Archives/edgar/data/12927/...,2017,1,10-K,BA,Boeing Co.,12927
2,18230,2017-02-15,\n10-K\n1\ncat_10-kx12312016.htm\n10-K\n\n\n\n...,https://www.sec.gov/Archives/edgar/data/18230/...,2017,1,10-K,CAT,Caterpillar Inc.,18230
3,93410,2017-02-23,\n10-K\n1\ncvx-123116x10kdoc.htm\n10-K\n\n\n\n...,https://www.sec.gov/Archives/edgar/data/93410/...,2017,1,10-K,CVX,Chevron Corp,93410
4,30554,2017-02-02,\n10-K\n1\ndd-12312016x10k.htm\n10-K\n\n\n\n\n...,https://www.sec.gov/Archives/edgar/data/30554/...,2017,1,10-K,DD,Dupont E I De Numours and Co.,30554
5,40545,2017-02-24,\n10-K\n1\nge10k2016.htm\n\n\n\n\n\n\n\n\n\n\n...,https://www.sec.gov/Archives/edgar/data/40545/...,2017,1,10-K,GE,General Electric Co.,40545
6,886982,2017-02-27,\n10-K\n1\nd308759d10k.htm\nFORM 10-K\n\n\nFor...,https://www.sec.gov/Archives/edgar/data/886982...,2017,1,10-K,GS,Goldman Sachs Group Inc.,886982
7,354950,2017-03-23,\n10-K\n1\nhd-01292017x10xk.htm\n10-K\n\n\n\n\...,https://www.sec.gov/Archives/edgar/data/354950...,2017,1,10-K,HD,Home Deport Inc.,354950
8,51143,2017-02-28,\n10-K\n1\na2230222z10-k.htm\n10-K\n\n\n\n\n\n...,https://www.sec.gov/Archives/edgar/data/51143/...,2017,1,10-K,IBM,International Business Machines Co.,51143
9,50863,2017-02-17,\n10-K\n1\na10kdocument12312016q4.htm\n10-K\n\...,https://www.sec.gov/Archives/edgar/data/50863/...,2017,1,10-K,INTC,Intel Co.,50863


In [158]:
# options to save files pickle file vs. csv
save_as_pickle("latest_10_k.txt", latest)
latest.to_csv("latest_10_k.csv")

In [160]:
# Reading Pickle file in:
new_data = read_data("latest_10_k.txt")

In [161]:
new_data

,CIK,file_date,full_text,url,year,quarter,report_type,Company,Name,CIK #
0,4962,2017-02-17,\n10-K\n1\nd321397d10k.htm\nFORM 10-K\n\n\nFor...,https://www.sec.gov/Archives/edgar/data/4962/0...,2017,1,10-K,AXP,American Expression Co.,4962
1,12927,2017-02-08,\n10-K\n1\na201612dec3110k.htm\n10-K\n\n\n\n\n...,https://www.sec.gov/Archives/edgar/data/12927/...,2017,1,10-K,BA,Boeing Co.,12927
2,18230,2017-02-15,\n10-K\n1\ncat_10-kx12312016.htm\n10-K\n\n\n\n...,https://www.sec.gov/Archives/edgar/data/18230/...,2017,1,10-K,CAT,Caterpillar Inc.,18230
3,93410,2017-02-23,\n10-K\n1\ncvx-123116x10kdoc.htm\n10-K\n\n\n\n...,https://www.sec.gov/Archives/edgar/data/93410/...,2017,1,10-K,CVX,Chevron Corp,93410
4,30554,2017-02-02,\n10-K\n1\ndd-12312016x10k.htm\n10-K\n\n\n\n\n...,https://www.sec.gov/Archives/edgar/data/30554/...,2017,1,10-K,DD,Dupont E I De Numours and Co.,30554
5,40545,2017-02-24,\n10-K\n1\nge10k2016.htm\n\n\n\n\n\n\n\n\n\n\n...,https://www.sec.gov/Archives/edgar/data/40545/...,2017,1,10-K,GE,General Electric Co.,40545
6,886982,2017-02-27,\n10-K\n1\nd308759d10k.htm\nFORM 10-K\n\n\nFor...,https://www.sec.gov/Archives/edgar/data/886982...,2017,1,10-K,GS,Goldman Sachs Group Inc.,886982
7,354950,2017-03-23,\n10-K\n1\nhd-01292017x10xk.htm\n10-K\n\n\n\n\...,https://www.sec.gov/Archives/edgar/data/354950...,2017,1,10-K,HD,Home Deport Inc.,354950
8,51143,2017-02-28,\n10-K\n1\na2230222z10-k.htm\n10-K\n\n\n\n\n\n...,https://www.sec.gov/Archives/edgar/data/51143/...,2017,1,10-K,IBM,International Business Machines Co.,51143
9,50863,2017-02-17,\n10-K\n1\na10kdocument12312016q4.htm\n10-K\n\...,https://www.sec.gov/Archives/edgar/data/50863/...,2017,1,10-K,INTC,Intel Co.,50863


Second Demonstration:

Financial Crisis 2008 Data
Survived: 1) BOA, 2) JP MORGAN, 3) Wells Fargo, 4) AIG, and 5) Citi.
merged_banks: 1) countrywide, 2) lehman brothers, 3) Wacovia, 4) BEAR STEARNS, and 5) Merrill Lynch

In [34]:
merged_banks=[25191, 806085, 36995, 777001, 65100]

In [46]:
survived = [70858, 19617, 72971, 5272, 831001]

In [44]:
merged_2008_qtr1 = putting_together(2008, 1, merged_banks, "10-K")

In [53]:
merged_2008_qtr1

,CIK,full_text,year,quarter,report_type,Company,Name,CIK #
0,25191,\n10-K\n1\na2182824z10-k.htm\n10-K\n\n\n\n\n\n...,2008,1,10-K,CFC,CountryWide,25191
1,806085,\n10-K\n1\na08-3530_110k.htm\n10-K\n\n\n\n\n\n...,2008,1,10-K,LEH,Lehman Brothers,806085
2,36995,\n10-K\n1\ng11813e10vk.htm\nWACHOVIA CORPORATI...,2008,1,10-K,WB,Wacovia,36995
3,777001,\n10-K\n1\nbe11750956-10k.txt\nANNUAL REPORT\n...,2008,1,10-K,BSC,Bear Stearns,777001
4,65100,\n10-K\n1\ny46644e10vk.htm\nFORM 10-K\n\n\n\n1...,2008,1,10-K,MER,Merill Lynch,65100


In [57]:
merged_2008_qtr1.to_csv("merged_2008_qtr1.csv",index = False)

In [47]:
survived_2008_qtr1 = putting_together(2008, 1, survived, "10-K")

In [55]:
survived_2008_qtr1

,CIK,full_text,year,quarter,report_type,Company,Name,CIK #
0,70858,\n10-K\n1\nd10k.htm\nFORM 10-K\n\n\nForm 10-K\...,2008,1,10-K,BAC,Bank of America,70858
1,19617,\n10-K\n1\nd10k.htm\nFORM 10-K\n\n\nForm 10-K\...,2008,1,10-K,JPM,J P Morgan Chase & Co.,19617
2,72971,\n10-K\n1\nf38267e10vk.htm\nFORM 10-K\n\n\n\ne...,2008,1,10-K,WFC,Wells Fargo,72971
3,5272,\n10-K\n1\ny44393e10vk.htm\nFORM 10-K\n\n\n\nF...,2008,1,10-K,AIG,American International Group,5272
4,831001,\n10-K\n1\nd10k.htm\nFORM 10-K\n\n\nForm 10-K\...,2008,1,10-K,C,Citi,831001


In [56]:
survived_2008_qtr1.to_csv("survived_2008_qtr1.csv",index = False)